In [1]:
import random as rd
import numpy as np
import binascii

from glob import glob

In [2]:
def euclid_extended(a, b):
    '''
    Realiza o cálculo do mdc
    entre a e b, retornando
    o mdc e x e y tais que
    ax + by = mdc(a, b).
    '''
    inverted = False
    if b > a:
        a, b = b, a
        inverted = True
        
    
    table = np.array([[a, b], [1, 0], [0, 1]])
    iteration = 0
    while table[0, (iteration + 1) % 2] != 0:
        a, b = table[0, iteration % 2], table[0, (iteration + 1) % 2]
        q = a // b
        r = a % b
        table[:, iteration % 2] -= table[:, (iteration + 1) % 2] * q
        iteration += 1
        
    lcd, x, y = table[:, iteration % 2]
    
    if inverted:
        return lcd, y, x
    else:
        return lcd, x, y

In [3]:
def is_prime(n):
    '''
    Recebe um número e verifica
    se ele é primo
    '''
    if n == 2:
        return True
    
    if n < 2 or n % 2 == 0:
        return False
    
    for i in range(3, int(n**(0.5)) + 1, 2):
        if n % i == 0:
            return False
        
    return True

In [4]:
# Buscando os primos de 10.000 até
# 19.999

if 'primos.npy' not in glob('*.npy'):
    primes = []
    a = 10000
    for i in range(5000):
        if is_prime(a + 2 * i + 1):
            primes.append(a + 2 * i + 1)

    primes = np.array(primes)
    with open('primos.npy', 'wb') as f:
        np.save(f, primos)

else:
    primes = np.load('primos.npy')

primes

array([10007, 10009, 10037, ..., 19991, 19993, 19997])

In [5]:
def choose_parameters(primes):
    '''
    Recebe um array de primos e
    retorna dois primos p e q,
    phi(n) e números e e d de
    modo que de = 1 (mod phi(n)),
    onde n = p*q
    '''
    
    rd.shuffle(primes)
    p, q = primes[:2]
    phi_n = (p - 1) * (q - 1)
    
    e = 2
    lcd, _, d = euclid_extended(phi_n, e)
    lcd2, _, _ = euclid_extended(1114111, e)
    while lcd != 1 or lcd2 != 1:
        e = rd.randint(3, phi_n)
        lcd, _, d = euclid_extended(phi_n, e)
        lcd2, _, _ = euclid_extended(1114111, e)
    
    return p, q, phi_n, e, d, lcd

In [6]:
p, q, phi_n, e, d, mdc = choose_parameters(primes)
print((e * d) % phi_n, e)

1 148881841


In [7]:
o_message = 'Hello World!'

def to_number(message):
    m = ''
    for letter in message:
        m += str(ord(letter)).zfill(4)
        
    return m

def to_text(m):
    message = ''
    for i in range(int(len(m) / 4)):
        message += chr(int(m[4 * i : 4 * i + 4]))
        
    return message

m = to_number(o_message)
message = to_text(m)
print(m)
print(message)

def str2num(string):
    return int(binascii.hexlify(string.encode("utf-8")), 16)

def num2str(number):
    return binascii.unhexlify(format(number, "x").encode("utf-8")).decode("utf-8")

def numfy(s, max_code = 0x110000):
    # 0x110000 is max value of unicode character
    number = 0
    for e in [ord(c) for c in s]:
        number = (number * max_code) + e
    return number

def denumfy(number, max_code = 0x110000):
    l = []
    while number != 0:
        l.append(chr(number % max_code))
        number = number // max_code
    return ''.join(reversed(l))

m = str2num(o_message)
message = num2str(m)
print(m)
print(message)

m = numfy(o_message)
message = denumfy(m)
print(m)
print(message)

def ConvertToInt(message):
    grd = 1
    num = 0
    message = message [::-1]
    for i in range(0,len(message),+1):
        num = num+ord(message[i])*grd
        grd *= 256
    return num

def ConvertToStr(num):
    st = ""
    while (num != 0):
        temp = num % 256
        st += chr(temp)
        chr(temp)
        num = num - temp
        num = num // 256
    st = st[::-1]
    return st

s = ConvertToInt(o_message)
n = ConvertToStr(s)
print(s)
print(n)

007201010108010801110032008701110114010801000033
Hello World!
22405534230753928650781647905
Hello World!
236347174792598260040102094117559934601249070937025207111779405266977
Hello World!
22405534230753928650781647905
Hello World!


In [8]:
p, q = 100000000000000003, 100000000000000013            
phi_n = (p - 1) * (q - 1)
e = 2
lcd, _, d = euclid_extended(phi_n, e)
while lcd != 1:
    e = rd.randint(3, phi_n)
    lcd, _, d = euclid_extended(phi_n, e)

(phi_n * _ + e * d) % phi_n

1

In [9]:
# def encrypt(message, phi_n, e):
#     phi_n = int(phi_n)
#     m = to_number(message)
#     encrypted = ''
#     encryptedn = ''
#     i = 0
#     number = ''
#     letters = 0
#     while i < len(m):
#         if int(number + m[i:i + 4]) < phi_n:
#             number += m[i:i + 4]
#             letters += 1
#             i += 4
#         else:
#             print(number)
#             number = int(number)
#             number = pow(number, e, phi_n)
#             number = str(number).zfill(letters * 4)
#             encrypted += to_text(number)
#             encryptedn += number
#             if (len(encryptedn) % 4) != 0:
#                 encryptedn = encryptedn.zfill((len(encryptedn) // 4 + 1) * 4)
                
#             number = ''
#             letters = 0
            
#     return encrypted, encryptedn, m

# def decrypt(encrypted, phi_n, d):
#     d = int(d)
#     phi_n = int(phi_n)
#     m = to_number(encrypted)
#     message = ''
#     i = 4
#     letters = 0
#     number = ''
#     while m != '':
#         aux = m[0:i]
#         aux = int(aux)
#         if aux < phi_n and i <= len(m):
#             number = m[0:i]
#             letters += 1
#             i += 4
#         else:
#             number = int(number)
#             number = pow(number, d, phi_n)
#             number = str(number).zfill(letters * 4)
#             message += to_text(number)
#             number = ''
#             m = m[4 * letters:]
#             letters = 0
#             i = 4
            
#     return message

# m = encrypt(o_message, phi_n, e)
# message = decrypt(m[0], phi_n, d)
# print(m[2])
# print(m[2] == to_number(o_message))
# print(m[1], len(m[1]))
# print(message)
# print(m[1] == message)

In [10]:
def encrypt(message, phi_n, e):
    phi_n = int(phi_n)
    m = ConvertToInt(message)
    m = (m * e) % phi_n
    encrypted = ConvertToStr(m)
    
    return encrypted

def decrypt(encrypted, phi_n, d):
    d = int(d)
    phi_n = int(phi_n)
    m = ConvertToInt(encrypted)
    m = (m * d) % phi_n
    message = ConvertToStr(m)
    
    return message

m = encrypt(o_message, phi_n, e)
message = decrypt(m, phi_n, d)
print(m)
print(message)
print(message == o_message)

'±_T:z]<¹Å?
Hello World!
True
